In [ ]:
# from data.unlabeled import TiktokenTokenizer, download_dataset
from data.loader import memmapDL, distributedMemmapDL

from model.gpt import GPT

from learner.callbacks import save_checkpoints
from learner.LLMLearner import LLMLearner

from fastai.text.all import *
from fastai.distributed import *
from fastai.callback.wandb import *
import wandb


/home/palaash/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'TiktokenTokenizer' from 'data.tokenizer' (/home/palaash/allmond/data/tokenizer.py)

In [10]:
model = GPT.from_hf('microsoft/Phi-3-mini-4k-instruct')

NameError: name 'GPT' is not defined

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct', trust_remote_code = True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaTokenizerFast(name_or_path='microsoft/Phi-3-mini-4k-instruct', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=

In [7]:
??tokenizer.encode

Signature:
tokenizer.encode(
    text: Union[str, List[str], List[int]],
    text_pair: Union[str, List[str], List[int], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = None,
    max_length: Optional[int] = None,
    stride: int = 0,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    **kwargs,
) -> List[int]
Docstring:
Converts a string to a sequence of ids (integer), using the tokenizer and vocabulary.

Same as doing `self.convert_tokens_to_ids(self.tokenize(text))`.

Args:
    text (`str`, `List[str]` or `List[int]`):
        The first sequence to be encoded. This can be a string, a list of strings (tokenized string using the
        `tokenize` method) or a list of integers (tokenized string ids using the `convert_tokens_to_ids`
        method).
    text_pair (`str

In [8]:
model

NameError: name 'model' is not defined

In [8]:
tokenizer.eos_token_id

32000

In [13]:
model = AutoModelForCausalLM.from_pretrained('microsoft/Phi-3-mini-4k-instruct', trust_remote_code = True) 

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.47it/s]


In [14]:
??model

Signature:       model(*args, **kwargs)
Type:            Phi3ForCausalLM
String form:    
Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
 

In [1]:
from data.unlabeled import download_dataset
from data.loader import memmapDL, distributedMemmapDL

from model.gpt import GPT

from learner.callbacks import save_checkpoints
from learner.LLMLearner import LLMLearner

from fastai.text.all import *
from fastai.distributed import *
from fastai.callback.wandb import *
import wandb

# os.environ['NCCL_P2P_LEVEL']='NVL'
#If you see  see an error like this after waiting for a long time. 
# RuntimeError: Exception occured in `DistributedTrainer` when calling event `before_fit`:  DDP expects same model across all ranks, but Rank 1 has 221 params, while rank 0 has inconsistent 0 params.
#This seems like a NVIDIA problem due to the "asynchronous nature of CUDA kernels". Well then how can i make them synchronous? Asynchronous anyways does not look like a sound choice for parallel CUDA operations.
#Try uncommenting the above line

#________________________________________wandb____________________________________________
log_wandb = False #set to False if you dont want to log progress to W&B

project = 'tinylm'
dataset = "wikisimple"
mode = 'scratch'
# ________________________________________hyperparams and settings_________________________

bs=1 #each GPU gets bs = 20, works good for a 24GB GPU
# block_size = 512
block_size = 100
valid_sampler_size = 1000 #how many samples to use for validation. This is only used to check if validation loss is better than best_valid_loss, so that a checkpoint can be saved. Karpathy uses 200 random points
validate_every = 1000 #1000 iterations, each iteration is bs*total_GPUs inputs

#________________________________________Model_____________________________________________


model = GPT(block_size=block_size, n_layer = 200) 

#________________________________________data______________________________________________

train_path, valid_path = rank0_first(lambda: download_dataset(dataset = dataset, encoder = model.tokenizer)) #check if data exists, download only for rank0 GPU. 
train_dl = memmapDL(train_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype())
valid_dl = memmapDL(valid_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype(), 
                      sample_size = valid_sampler_size)

if num_distrib(): #distributed training
    train_dl, valid_dl = distributedMemmapDL(train_dl), distributedMemmapDL(valid_dl)
    
if not rank_distrib(): print(f'training {str(model)} on {train_dl.n} tokens') #print only for rank0
    

dls = DataLoaders(train_dl, valid_dl)
dls.c = model.head.out_features

#________________________________________Trainer____________________________________________

check_and_save_model = save_checkpoints(dir = Path('checkpoints'), 
                                                model_name = 'gpt', 
                                                checkpoint_name = str(model), 
                                                every_iters = validate_every)
cbs = [check_and_save_model]
if log_wandb:
    cbs.append(WandbCallback())
    wandb.init(project=project, name = f"{id}_{str(model)}_{mode}")


learn = LLMLearner(dls, 
                model, 
                opt_func = partial(OptimWrapper, opt=torch.optim.AdamW),
                loss_func=CrossEntropyLossFlat(), 
                metrics=[accuracy, Perplexity()],
                cbs = cbs,
                path = check_and_save_model.path,
                model_dir=check_and_save_model.model_dir, 
                ).to_bf16()


/home/palaash/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting num_proc from 18 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 100%|██████████| 205328/205328 [00:01<00:00, 109350.13 examples/s]


training GPT-1.8B on 53902209 tokens


In [1]:
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP


/home/palaash/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kwargs = {
                        "sharding_strategy": fsdp_plugin.sharding_strategy,
                        "cpu_offload": fsdp_plugin.cpu_offload,
                        "auto_wrap_policy": fsdp_plugin.auto_wrap_policy,
                        "mixed_precision": fsdp_plugin.mixed_precision_policy,
                        "sync_module_states": fsdp_plugin.sync_module_states,
                        "backward_prefetch": fsdp_plugin.backward_prefetch,
                        "forward_prefetch": fsdp_plugin.forward_prefetch,
                        "use_orig_params": fsdp_plugin.use_orig_params,
                        "param_init_fn": fsdp_plugin.param_init_fn,
                        "ignored_modules": fsdp_plugin.ignored_modules,
                        "limit_all_gathers": fsdp_plugin.limit_all_gathers,
                        "device_id": self.device,
                    }

NameError: name 'fsdp_plugin' is not defined